In [53]:
import math

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
block_size = 3 # how many characters do we take to predict the next one?

X, Y = [], []
context = [0] * block_size
for w in words[:5]:
    for ch in w+'.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        
        print(''.join(itos[i] for i in context), f'---> {itos[ix]}')
        context = context[1:]+[ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
ma. ---> o
a.o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ia. ---> a
a.a ---> v
.av ---> a
ava ---> .
va. ---> i
a.i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
la. ---> s
a.s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [12]:
C = torch.randn(27, 2)
C

tensor([[ 0.2677, -1.4295],
        [ 0.1604,  0.0037],
        [-0.7823,  1.2045],
        [-0.9336,  0.2730],
        [-0.2526, -1.2344],
        [-0.9823,  0.2718],
        [ 0.1416, -0.9785],
        [-0.8715,  0.7930],
        [ 0.8436, -0.3729],
        [-2.9927,  0.5206],
        [-0.2309,  1.8971],
        [-0.0326,  1.1610],
        [ 0.1670, -0.5818],
        [ 0.3613,  1.3046],
        [-0.7404,  2.2121],
        [-0.3249, -0.0383],
        [ 1.0778,  0.4838],
        [-0.5800, -1.5607],
        [-0.8577, -1.1597],
        [ 0.0088, -0.0777],
        [ 0.9126,  0.1410],
        [ 1.9503, -0.4797],
        [-1.3162,  1.3912],
        [-0.0961,  1.7952],
        [-2.2434,  1.2519],
        [-1.3728,  0.0604],
        [ 1.9538, -0.1571]])

In [14]:
C[X].shape

torch.Size([32, 3, 2])

In [16]:
X[23][1]

tensor(12)

In [17]:
C[X][23][1]

tensor([ 0.1670, -0.5818])

In [18]:
C[12]

tensor([ 0.1670, -0.5818])

In [19]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [23]:
W = torch.randn(6, 100)
b = torch.randn(100)

torch.Size([6, 100])

In [25]:
emb @ W + b

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [26]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [31]:
torch.cat(emb.unbind(1), 1).shape

torch.Size([32, 6])

In [44]:
nums = torch.arange(18).view(2, 3, 3)
nums

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]])

In [48]:
nums.view(-1, 2, 3)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]]])

In [51]:
nums.storage() # torch stores flat tensors http://blog.ezyang.com/2019/05/pytorch-internals/

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [55]:
h = torch.tanh(emb.view(32, -1) @ W + b)

In [52]:
print(W.shape, b.shape)

torch.Size([6, 100]) torch.Size([100])


In [ ]:
# 6, 100
#    100

# careful with torch broadcasting

In [58]:
W2 = torch.randn(100, 27)
b2 = 27

logits = h @ W2 +b2

In [59]:
logits.shape

torch.Size([32, 27])

In [60]:
counts = logits.exp()
probs = counts/counts.sum(1, keepdims=True)

In [62]:
probs.shape

torch.Size([32, 27])

In [70]:
loss = -probs[torch.arange(probs.shape[0]), Y].log().mean()
loss

tensor(16.1386)